In [3]:
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank
from lightfm.evaluation import auc_score
from lightfm.data import Dataset
import click
import os
import logging
from numpy.typing import NDArray
from typing import Any
import pickle
from tqdm import tqdm
# TODO set k to 500 and test execution time
# perform tuning up to loss function, epochs, for top_k_accuracy etc

In [4]:
def load_pickle(input_path) -> Any:
    with open(input_path, 'rb') as file:
        return pickle.load(file)

def save_pickle(object: Any, output_path: str):
    with open(output_path, 'wb') as file:
        pickle.dump(object, file)

In [5]:
input_filepath = "/home/adamgorski/Desktop/inzynierka/conseillify/conseillify-research/data/dataset"
output_filepath = "/home/adamgorski/Desktop/inzynierka/conseillify/conseillify-research/data/predictions/candidate_generation"
model_path = "/home/adamgorski/Desktop/inzynierka/conseillify/conseillify-research/models/candidate_generation"

In [6]:
train_interactions = load_npz(os.path.join(input_filepath, "train_interactions.npz"))
val_interactions = load_npz(os.path.join(input_filepath, "val_interactions.npz"))
test_interactions = load_npz(os.path.join(input_filepath, "test_interactions.npz"))
dataset = load_pickle(os.path.join(input_filepath, "dataset_lightfm"))

In [7]:
search_grid = {
    "no_components": [5, 10, 25],
    "loss": ['bpr', 'warp'],
    "epochs": [3, 8]
}
grid = {ls: {nc: [e for e in search_grid["epochs"]] for nc in search_grid["no_components"]} for ls in search_grid["loss"]}
model_precision_at_k = {ls: {nc: [] for nc in search_grid["no_components"]} for ls in search_grid["loss"]}


In [8]:
for ls, no_components in grid.items():
    for nc, epochs in no_components.items():
        for e in epochs:
            model = LightFM(no_components=nc, loss=ls)
            model.fit(train_interactions, epochs=e, verbose=True)
            prec = np.average(precision_at_k(model, val_interactions, k=500, num_threads=8))
            model_precision_at_k[ls][nc].append(prec)



Epoch: 100%|██████████| 8/8 [16:27<00:00, 123.38s/it]


In [6]:
model.fit(train_interactions, epochs=3, verbose=True, num_threads=8)

Epoch: 100%|██████████| 3/3 [01:35<00:00, 31.97s/it]


In [7]:
model_precision_at_k = precision_at_k(model, val_interactions, k=500, num_threads=8)

In [9]:
model_auc_score = auc_score(model, val_interactions , num_threads=8)


In [10]:
print(model_precision_at_k.shape)
print(model_auc_score.shape)

(5000,)
(5000,)


In [12]:
print(np.average(model_precision_at_k))
print(np.average(model_auc_score))

0.035140004
0.95094025


In [8]:
print(np.average(model_precision_at_k))

0.0154712
